<a href="https://colab.research.google.com/github/TapasKumarDutta1/Multihead_attention/blob/master/brats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
import h5py
from tensorflow.keras.optimizers import *
import cv2
from tensorflow.keras.utils import to_categorical
import glob, os
from matplotlib import pyplot as plt
import h5py
from sklearn.metrics import accuracy_score
import numpy as np
from tqdm import tqdm
import time
import gc
from tensorflow.keras.applications import *
from tensorflow.keras.layers import *
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.activations import sigmoid
from tensorflow.keras.optimizers import *
import tensorflow as tf
def block(inputs,f):
    a=Conv2D(f,3,padding='same')(inputs)
    a=BatchNormalization()(a)
    a=ReLU()(a)
    b=Conv2D(f,3,padding='same')(a)
    b=BatchNormalization()(b)
    b=ReLU()(b)
    c=Conv2D(f,3,padding='same')(b)
    c=BatchNormalization()(c)
    c=ReLU()(c)
    d=Conv2D(f,3,padding='same')(c)
    d=BatchNormalization()(d)
    d=ReLU()(d)
    mid=Concatenate()([a,b,c,d])
    
    mid=Conv2D(2*f,1,padding='same')(mid)
    mid=BatchNormalization()(mid)
    mid=ReLU()(mid)
    
    x=Conv2D(f*2,1)(inputs)
    x=BatchNormalization()(x)
    x=ReLU()(x)
    
    x=Add()([mid,x])
    
    y=Conv2D(f*2,1)(x)
    y=BatchNormalization()(y)
    y=ReLU()(y)
    return y



def Global_attention_block(C_A):
    
    x=Lambda(lambda x: K.mean(x,axis=-1,keepdims=True))  (C_A)
    y=Lambda(lambda x: K.max(x,axis=-1,keepdims=True))  (C_A)
    
    x=Concatenate()([x,y])
    x=Activation('relu') (x)
    x=Conv2D(1,1, padding='same') (x)
    x=Activation('sigmoid') (x)
    S_A=Multiply()([x,C_A])
    return S_A

def self_attention(inp):
    shp=inp.shape
    a=Conv2D(shp[3]//8,1, padding='same') (inp)
    a=Activation('relu') (a)
    b=Conv2D(shp[3]//8,1, padding='same') (inp)
    b=Activation('relu') (b)
    c=Conv2D(shp[3]//8,1, padding='same') (inp)
    c=Activation('relu') (c)
    
    a=Reshape(( shp[1]*shp[2], shp[3]//8))(a)
    b=Reshape(( shp[1]*shp[2], shp[3]//8))(b)
    b=K.permute_dimensions(b, (0, 2, 1))
    c=Reshape(( shp[1]*shp[2], shp[3]//8))(c)
    inter=K.batch_dot(a,b)
    inter=Activation('softmax') (inter)
    out=K.batch_dot(inter,c)
    out=Reshape(( shp[1],shp[2], shp[3]//8))(out)
    out=Conv2D(shp[3],1, padding='same') (out)
    out=Activation('relu') (out)
    return out

def channel_attention(inputs):
    shape=K.int_shape(inputs)
    x=MaxPooling2D(pool_size=(shape[1],shape[2])) (inputs)
    x=Conv2D(shape[3]//8,1, padding='same',kernel_initializer='he_normal', use_bias=False) (x)
    x=Activation('relu') (x)
    x=Conv2D(shape[3],1, padding='same',kernel_initializer='he_normal', use_bias=False) (x)
    x=Activation('sigmoid') (x)
    x=Multiply()([x,inputs])
    return x

def load_model():   
  K.clear_session() 
  inputs = Input(shape=(224,224,3))
  x=Conv2D(16,3,padding='same')(inputs)
  x=BatchNormalization()(x)
  x=ReLU()(x)
  x=MaxPooling2D()(x)
  
  x=Conv2D(16,3,padding='same')(x)
  x=BatchNormalization()(x)
  x=ReLU()(x)
  x=MaxPooling2D()(x)
    
  a1=block(x,32)
  x=MaxPooling2D()(a1)

  a2=block(x,64)
  x=MaxPooling2D()(a2)
    
  a3=block(x,128)

#   a31=self_attention(a3)
#   a32=Global_attention_block(a3)
#   a3=Add()([a31,a32])
#   x=channel_attention(a3)
    
  x=GlobalMaxPooling2D()(a3)
    
  x=Dropout(0.5)(x)
  x=Dense(2,activation='softmax')(x)
  model = Model(inputs=inputs, outputs=x)
  
  return model




from tensorflow import keras 
from tensorflow.keras.activations import sigmoid
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import *
from tensorflow.keras.applications import *
from tensorflow import keras 
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import *
from tensorflow.keras.applications import *
import pandas as pd
import imgaug as ia
from imgaug import augmenters as iaa

def rotate_image(image, angle):
  image_center = tuple(np.array(image.shape[1::-1]) / 2)
  rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
  result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
  return result
def Hflip( images):
		seq = iaa.Sequential([iaa.Fliplr(1.0)])
		return seq.augment_images(images)
def Vflip( images):
		seq = iaa.Sequential([iaa.Flipud(1.0)])
		return seq.augment_images(images)
def noise(images):
    ls=[]
    for i in images:
        x = np.random.normal(loc=0, scale=0.05, size=(299,299,3))
        ls.append(i+x)
    return ls
def rotate(images):
    ls=[]
    for angle in range(-15,15,5):
        for image in images:
            ls.append(rotate_image(image,angle))
    return ls
class DataGenerator(tf.keras.utils.Sequence):
  def __init__(self, images, labels, batch_size=64, image_dimensions = (96 ,96 ,3), shuffle=False, augment=False):
    self.labels       = labels              # array of labels
    self.images = images        # array of image paths
    self.batch_size   = batch_size          # batch size
    self.on_epoch_end()

  def __len__(self):
    return int(np.floor(self.labels.shape[0] / self.batch_size))

  def on_epoch_end(self):
    self.indexes = np.arange(self.labels.shape[0])

  def __getitem__(self, index):
		# selects indices of data for next batch
    indexes = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]
    # select data and load images
    labels = self.labels.loc[indexes]
    img = [self.images[k].astype(np.float32) for k in indexes]
    imgH=Hflip(img)
    imgV=Vflip(img)
    imgR=rotate(img)
    images=[]
    images.extend(imgH)
    images.extend(imgV)
    images.extend(imgR)
    lbl=labels.copy()
    labels=pd.DataFrame()
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    #images = np.array([preprocess_input(img) for img in images])
    return np.asarray(images), labels.values


best_accuracy_last={}
final_accuracy_last={}
history_last={}
answers_last={}
predictions_last={}
predictions_last_best={}
times_last={}


#shuffle samples
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]



#change targets
def change(img):
    resized = cv2.resize(img, (224,224), interpolation = cv2.INTER_AREA )
    return resized




#get train and test splits
def get_trn_tst(df,tst_fold):
  idx=np.asarray(df['fold'])
  y=np.asarray(df['label'])
  y-=1
  img=np.asarray(df['images'])
  img1=[]
  for i in range(len(img)):
        img1.append(change(img[i]))
  img1=np.asarray(img1)
  del([img])
  gc.collect()
  trn_y=np.asarray(y[(idx!=tst_fold)])
  trn_img=np.asarray(img1[(idx!=tst_fold)])
  tst_y=np.asarray(y[(idx==tst_fold)])
  tst_img=img1[idx==tst_fold]
  trn_img=np.repeat(trn_img.reshape((trn_img.shape[0],224,224,1)),3,axis=3)
  tst_img=np.repeat(tst_img.reshape((tst_img.shape[0],224,224,1)),3,axis=3)
  return (trn_img.copy(),trn_y.copy()),(tst_img.copy(),tst_y.copy())

from pathlib import Path
def get_trn_tst(index):
    index=str(index-1)
    test=[]
    for i in np.load('/kaggle/input/2020-stratified-split/test_'+index+'.npy',allow_pickle=True):
        if Path("../input/brats2020-classification/"+i+"_LGG1.npy").is_file():
            test.append("../input/brats2020-classification/"+i+"_LGG1.npy")
        if Path("../input/brats2020-classification/"+i+"_LGG2.npy").is_file():
            test.append("../input/brats2020-classification/"+i+"_LGG2.npy")
        if Path("../input/brats2020-classification/"+i+"_LGG3.npy").is_file():
            test.append("../input/brats2020-classification/"+i+"_LGG3.npy")
        if Path("../input/brats2020-classification/"+i+"_HGG.npy").is_file():
            test.append("../input/brats2020-classification/"+i+"_HGG.npy")
    train=[]
    for i in np.load('/kaggle/input/2020-stratified-split/train_'+index+'.npy',allow_pickle=True):
        if Path("../input/brats2020-classification/"+i+"_LGG1.npy").is_file():
            train.append("../input/brats2020-classification/"+i+"_LGG1.npy")
        if Path("../input/brats2020-classification/"+i+"_LGG2.npy").is_file():
            train.append("../input/brats2020-classification/"+i+"_LGG2.npy")
        if Path("../input/brats2020-classification/"+i+"_LGG3.npy").is_file():
            train.append("../input/brats2020-classification/"+i+"_LGG3.npy")
        if Path("../input/brats2020-classification/"+i+"_HGG.npy").is_file():
            train.append("../input/brats2020-classification/"+i+"_HGG.npy")
    return train,test
def unison_shuffled_copies(a):
    p = np.random.permutation(len(a))
    return list(np.asarray(a)[p])
class DataGenerator(tf.keras.utils.Sequence):
  def __init__(self, images, batch_size=64, image_dimensions = (96 ,96 ,3), shuffle=False, augment=False):
    self.images = images        # array of image paths
    self.batch_size   = batch_size          # batch size
    self.augment=augment
    self.on_epoch_end()

  def __len__(self):
    return int(np.ceil(len(self.images) / self.batch_size))

  def on_epoch_end(self):
    self.indexes = np.arange(len(self.images))

  def __getitem__(self, index):
    max=(index + 1) * self.batch_size
    if max>len(self.images):
        max=len(self.images)
    indexes = self.images[index * self.batch_size : max]
    img = [  ]
    for k in indexes:
        img.append(np.repeat(cv2.resize(np.load(k), (224,224), interpolation = cv2.INTER_AREA ).reshape((224,224,1)),3,axis=-1).astype(np.float32))
    imgH=Hflip(img)
    imgV=Vflip(img)
    imgR=rotate(img)
    images=img
    if self.augment:
        images.extend(imgH)
        images.extend(imgV)
        images.extend(imgR)
    
    labels=[]
    for i in indexes:
        cls1=i.split('_')[-1].split('.')[0]
        if cls1[-1]=='G':
            labels.append([1,0])
        else:
            labels.append([0,1])
    if self.augment:
        labels*=9
        labels=np.stack(labels)
    return np.asarray(images), np.asarray(labels)
def upd(dk,data):
    if dk==0:
        dk=data
    else:
        for ky in data.keys():
            dk[ky].extend(data[ky])
    return dk
for index in range(1,6):
  best_accuracy_last={}
  final_accuracy_last={}
  history_last={}
  answers_last={}
  predictions_last={}
  predictions_last_best={}
  times_last={}
  epoch=50
  pre_acc=0
  best=0
  fold='fold_'+str(index)
  trn,tst=get_trn_tst(index)
  history_last[fold]=0






  trn=unison_shuffled_copies(trn)
  tst=unison_shuffled_copies(tst)



  model=load_model()


  
  #compiling the model
  model.compile(optimizer=Adam(1e-3,decay=1e-4), 
                     loss='categorical_crossentropy', 
                     metrics=['accuracy'])
  train_data = DataGenerator(trn, batch_size=4, augment=True)
  test_data = DataGenerator(tst, batch_size=4, augment=False)
  ln=len(trn)
  # del([trn_x,trn_y,trn,tst])
  # gc.collect()
  #fitting the model
  #timing
  start=time.time()
  print('training')
  hist=model.fit_generator(train_data,epochs=50,steps_per_epoch=ln//4)
  history_last[fold]=upd(history_last[fold],hist.history)

  end=time.time()
  times_last[fold]=end-start

  #getting the prediction 
  pre=model.predict(test_data)
  



  #select the maximum position
  pre=np.argmax(pre,1)
  predictions_last[fold]=pre

  
  labels=[]
  for i in tst:
        cls1=i.split('_')[-1].split('.')[0]
        if cls1[-1]=='G':
            labels.append(0)
        else:
            labels.append(1)
  
  #getting the accuracy
  new_acc=accuracy_score(pre,labels)

  


  #storing the predictions
  final_accuracy_last[fold]=new_acc








  #storing the answers
  answers_last[fold]=labels
    
    
    
    
  #freeing memory
  del([train_data,test_data,labels])
  gc.collect()
  model.save_weights('weights.hdf5')
  plt.plot(history_last[fold]['loss'])
  plt.show()
  print(new_acc)
  index=str(index)
  type='final_3535'
  model1='final'
  np.save("best_accuracy_all_fold_"+index+"_"+model1+"_"+type+".npy",best_accuracy_last)
  np.save('final_accuracy_all_fold'+index+"_"+model1+"_"+type+".npy",final_accuracy_last)
  np.save('history_all_fold_'+index+"_"+model1+"_"+type+".npy",history_last)
  np.save('answers_all_fold_'+index+"_"+model1+"_"+type+".npy",answers_last)
  np.save('predictions_all_fold_'+index+"_"+model1+"_"+type+".npy",predictions_last)
  np.save('predictions_all_best_fold_'+index+"_"+model1+"_"+type+".npy",predictions_last_best)
  np.save('times_all_fold_'+index+"_"+model1+"_"+type+".npy",times_last)
  del([model])
  gc.collect()